In [17]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import scipy
import ipywidgets as iw
import time
import scipy.sparse as sp




## Assignment 5:

I have implemented the steps for multiresolutional mesh editing in 2 separate files. 
- `mesh_deformer.py` has regular **regular sparse solving** (`spsolve`)
- `cholesky_mesh_deformer.py`   using **Cholesky factorization**

## **Step 1: Surface Smoothing**

`extract_smooth_surface`
  - Minimize thin plate energy  v^T L_w M_inverse L_w v  subject to the constraints
  - Solve linear system : either using Cholesky factorization for high efficiency or directly solving for each coordinate sp.solve(Aff,rhs)
  - Smoothed surface B is made

## **Step 2: Deformation on Smoothed Surface**
  - handle deformations are applied to B'
  - The free vertices are recomputed to smoothly accommodate the new handle positions.
  - Deformation maintains global smoothness 
-**Implementation**:
  - System matrices (`Aff`, `Afc`) are reused.
  - Solve for new free vertex positions based on updated handles.

## **Step 3: Detail Preservation**

`extract_details`, `apply_details`

  - Fine geometric details (small displacements) are **extracted** from the original mesh.
  - Details are stored in the **local orthonormal frames** at each vertex.
  - After deformation, the stored fine details are **reapplied** to the deformed smoothed surface to reconstruct the final detailed mesh.
- **Implementation**:
  - Local frames  computed per vertex
  - Displacements  projected onto the frames to extract detail coefficients a1 a2 a3
  - After deformation, coefficients are re-projected onto new frames



In [26]:
import importlib
import mesh_deformer
importlib.reload(mesh_deformer)
from mesh_deformer import mesh_deformer

import cholesky_mesh_deformer
importlib.reload(cholesky_mesh_deformer)
from cholesky_mesh_deformer import cholesky_mesh_deformer

In [27]:
'''Using Mesh Deformer without Cholesky'''


v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

deformer = mesh_deformer(v, f, labels)
B = deformer.extract_smooth_surface(v)
a1, a2, a3 = deformer.encode_details(v, B)

def position_deformer(target_pos):
    Bprime = deformer.extract_smooth_surface(target_pos)
    
    Sprime = deformer.apply_details(Bprime, a1, a2, a3)

    return Sprime

#-------------------------------------------------------

handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x,y,z]])
    v_slice = v[slices] + displacement;
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

pos_f.deformer = position_deformer

p = mp.plot(handle_vertex_positions, f, c=labels)

def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i: iw.FloatSlider(min=-1, max=1, value=0) for i in 'xyz'}
    rotate_widget = {a: iw.FloatSlider(min=-90, max=90, value=0, step=1) for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,
         translate_widget['y'].value,
         translate_widget['z'].value,
         rotate_widget['α'].value,
         rotate_widget['β'].value,
         rotate_widget['γ'].value) = pos_f_saver[segment_widget.value - 1]

    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

iw.interact(pos_f, **widgets_wrapper())



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [20]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

cholesky_deformer =  cholesky_mesh_deformer(v, f, labels)
B = cholesky_deformer.extract_smooth_surface(v)
a1, a2, a3 = cholesky_deformer.encode_details(v, B)


def position_deformer(target_pos):
    Bprime = cholesky_deformer.extract_smooth_surface(target_pos)
    
    Sprime = cholesky_deformer.apply_details(Bprime, a1, a2, a3)

    return Sprime


#-------------------------------------------------
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x,y,z]])
    v_slice = v[slices] + displacement;
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

pos_f.deformer = position_deformer

p = mp.plot(handle_vertex_positions, f, c=labels)

def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i: iw.FloatSlider(min=-1, max=1, value=0) for i in 'xyz'}
    rotate_widget = {a: iw.FloatSlider(min=-90, max=90, value=0, step=1) for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,
         translate_widget['y'].value,
         translate_widget['z'].value,
         rotate_widget['α'].value,
         rotate_widget['β'].value,
         rotate_widget['γ'].value) = pos_f_saver[segment_widget.value - 1]

    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

iw.interact(pos_f, **widgets_wrapper())



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

### With Cholesky we can see the rendering of the movement of the handles is faster than that of solving a normal 

## Moving segment 2 handle z to -0.1:

-**Without cholesky FPS1.2827241446908275**
- refactorizes the Aff matrix every frame to solve with rhs.
- this is slower
  
-**With cholesky:   FPS 3.102651561384626**
- the facrotization is done once! and factor(rhs[:, i]) at each step is fast
- much faster'''



## Four meshes showing the transformations step by step: hand, woody, bumpy plane and cactus

In [12]:
import numpy as np
from scipy.spatial.transform import Rotation
import meshplot as mp

handle_vertex_positions = v.copy()

# maually defined transformations!
transformations = [
    (1, 0.2, 0.0, 0.0, 45, 0, 0),
    (2, 0.0, 0.2, 0.0, 0, 45, 0),
    (3, 0.0, 0.0, 0.2, 0, 0, 45),
]
for s, x, y, z, α, β, γ in transformations:
    if not np.any(labels == s):
        print(f"Skipping label {s} (no vertices)")
        continue

    slices = (labels == s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x, y, z]])
    v_slice = v[slices] + displacement
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center

# Step 3: Smooth deformed mesh and restore
Bprime = deformer.extract_smooth_surface(handle_vertex_positions)
Sprime = deformer.apply_details(Bprime, a1, a2, a3)

# Step 4: Plot in 2x2 grid
p= mp.subplot(v, f, c=labels, s=[2, 2, 0])      
mp.subplot(B, f,data=p, c=labels, s=[2, 2, 1])        
mp.subplot(Bprime, f,data=p, c=labels, s=[2, 2, 2])   
mp.subplot(Sprime, f,data=p, c=labels, s=[2, 2, 3])   


In [14]:
import meshplot as mp
from scipy.spatial.transform import Rotation

# Step 0: Read and normalize
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

deformer = mesh_deformer(v, f, labels)

# Step 1: Smooth and encode details on original mesh
B = deformer.extract_smooth_surface(v)
a1, a2, a3 = deformer.encode_details(v, B)

# Step 2: Apply manual transformations
handle_vertex_positions = v.copy()

print(np.unique(labels))

transformations = [
    (0, 0.2, 0.0, 0.0, 30, 0, 0),  
    (1, 0.0, 0.2, 0.0, 0, 30, 0), 
    (2, 0.2, 0.0, 0.0, 30, 0, 0),  
    (3, 0.0, 0.2, 0.0, 0, 30, 0),  
    (4, 0.2, 0.0, 0.0, 30, 0, 0), 
   
]

for s, x, y, z, α, β, γ in transformations:
    if not np.any(labels == s):
        print(f"Skipping label {s} (no vertices)")
        continue

    slices = (labels == s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x, y, z]])
    v_slice = v[slices] + displacement
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center

# Step 3: Smooth deformed mesh and restore
Bprime = deformer.extract_smooth_surface(handle_vertex_positions)
Sprime = deformer.apply_details(Bprime, a1, a2, a3)


p=mp.subplot(v, f, c=labels, s=[2, 2, 0])       # Original mesh
mp.subplot(B, f,data=p,  c=labels, s=[2, 2, 1])        # Smoothed original
mp.subplot(Bprime, f,data=p,  c=labels, s=[2, 2, 2])   # Sm
mp.subplot(Sprime, f,data=p,  c=labels, s=[2, 2, 3])   # Sm


[0 1 2 3 4 5]


In [15]:



v, f = igl.read_triangle_mesh('data/cactus.off')
labels = np.load('data/cactus.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
import numpy as np
from scipy.spatial.transform import Rotation
import meshplot as mp


deformer = mesh_deformer(v, f, labels)

print(np.unique(labels))

# Step 1: Smooth and encode details on original mesh BEFORE deformation
B = deformer.extract_smooth_surface(v)
a1, a2, a3 = deformer.encode_details(v, B)

# Step 2: Apply manual transformations
handle_vertex_positions = v.copy()

transformations = [
    (1, 0.2, 0.0, 0.0, 10, 0, 0),
    (2, 0.3, 0, 0.0, 0, 0, 0),
    (3, 0.0, 0.0, 0.2, 0, 0, 0),
]

for s, x, y, z, α, β, γ in transformations:
    if not np.any(labels == s):
        print(f"Skipping label {s} (no vertices)")
        continue

    slices = (labels == s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x, y, z]])
    v_slice = v[slices] + displacement
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center

# Step 3: Smooth deformed mesh and restore
Bprime = deformer.extract_smooth_surface(handle_vertex_positions)
Sprime = deformer.apply_details(Bprime, a1, a2, a3)


p=mp.subplot(v, f, c=labels, s=[2, 2, 0])       # original mesh
mp.subplot(B, f,data=p,  c=labels, s=[2, 2, 1])        # smoothed original
mp.subplot(Bprime, f,data=p, c=labels, s=[2, 2, 2])   # smoothed transformed
mp.subplot(Sprime, f,data=p, c=labels, s=[2, 2, 3])   # S transformed


[0 1 2 3 4]


In [16]:
import numpy as np
from scipy.spatial.transform import Rotation
import meshplot as mp

v, f = igl.read_triangle_mesh('data/bumpy_plane.off')
labels = np.load('data/bumpy_plane.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

deformer = mesh_deformer(v, f, labels)

print(np.unique(labels))

# Ssmooth and encode details on original mesh BEFORE deformation
B = deformer.extract_smooth_surface(v)
a1, a2, a3 = deformer.encode_details(v, B)

# apply manual transformations
handle_vertex_positions = v.copy()

transformations = [
    (1, 0.2, 0.0, 0.0, 45, 0, 0),
    (2, 0.0, 0.2, 0.0, 0, 45, 0),
    (3, 0.0, 0.0, 0.2, 0, 0, 45),
]

for s, x, y, z, α, β, γ in transformations:
    if not np.any(labels == s):
        print(f"Skipping label {s} (no vertices)")
        continue

    slices = (labels == s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    displacement = np.array([[x, y, z]])
    v_slice = v[slices] + displacement
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center

Bprime = deformer.extract_smooth_surface(handle_vertex_positions)
Sprime = deformer.apply_details(Bprime, a1, a2, a3)

p=mp.subplot(v, f, c=labels, s=[2, 2, 0])       
mp.subplot(B, f,data=p, c=labels, s=[2, 2, 1])        
mp.subplot(Bprime, f,data=p, c=labels, s=[2, 2, 2])   
mp.subplot(Sprime, f,data=p, c=labels, s=[2, 2, 3])   


[0 1 2 3 4]
